In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('..')

In [3]:
from src.experiments import MultimodalVAEExperiment
import yaml
import torch
from nflows.utils import torchutils
from torch.distributions import Categorical
from src.datamodules import MultimodalMNISTDataModule
from src.datamodules.multimodal_mnist import MultimodalMNIST
import torch.nn.functional as F

In [4]:
# Load config file
with open("configs/multimodal.yaml", "r") as file:
    try:
        hparams = yaml.safe_load(file)
    except yaml.YAMLError as exc:
        print(exc)

FileNotFoundError: [Errno 2] No such file or directory: 'configs/multimodal.yaml'

In [9]:
dm = MultimodalMNISTDataModule('data', batch_size=256)

In [10]:
dm.prepare_data()
dm.setup()

In [11]:
dl = dm.train_dataloader()

In [12]:
len(dl)

195

In [13]:
len(dm.val_dataloader())

39

In [8]:
batch = next(iter(dl))

In [9]:
batch['data'][0].shape, batch['data'][1].shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [40]:
logits = torch.randn(64, 3, 5)
target = torch.randint(5, (64,), dtype=torch.int64)

In [26]:
logits.shape, target.shape

(torch.Size([64, 5]), torch.Size([64]))

In [41]:
dist = Categorical(logits=logits)

In [18]:
dist.sample().shape

torch.Size([64])

In [25]:
dist.log_prob(target).mean()

tensor(-2.1742)

In [24]:
F.cross_entropy(logits, target)

tensor(2.1742)

In [33]:
dist.sample(torch.Size([7])).shape

torch.Size([7, 64])

In [34]:
samples = dist.sample(torch.Size([7]))

In [35]:
samples.permute(1, 0).shape

torch.Size([64, 7])

In [39]:
torch.argmax(logits, dim=-1).shape

torch.Size([64])